Importing Libraries

In [ ]:
# Importing Libs..

import os
from glob import glob

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

import tensorflow
import keras

from keras.applications import VGG16
from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import InceptionResNetV2
from keras.applications import Xception
from keras.applications import DenseNet201

from keras.models import Model, load_model
from keras.preprocessing import image
from keras.models import Model, Input
from keras.layers import BatchNormalization, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense

from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint
from keras import backend as K
from keras.utils import plot_model

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import cv2
import seaborn as sns
import gc

from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import roc_auc_score,precision_recall_fscore_support
import scipy

In [ ]:
import keras.backend as k
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.compat.v1.keras.backend import clear_session
from tensorflow.compat.v1.keras.backend import get_session


## Load  Dataset

In [ ]:
#D:\RahanaMSc\Projects\DR_F\scripts01\input

y = np.load('C:/Users/fredd/Downloads/aptos2019-blindness-detection/y_train.npy')
x = np.load('C:/Users/fredd/Downloads/aptos2019-blindness-detection/X_train_256.npy')
x_test_pred = np.load('C:/Users/fredd/Downloads/aptos2019-blindness-detection/X_test_256.npy')

In [ ]:
test = pd.read_csv('C:/Users/fredd/Downloads/aptos2019-blindness-detection/test.csv')

In [ ]:
from sklearn.model_selection import train_test_split
x_train_o, x_test, y_train_o, y_test = train_test_split(x, y,test_size = 0.10,random_state=42)

In [ ]:
x_train_o.shape,x_test.shape, y_train_o.shape, y_test.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train_o, y_train_o,test_size = 0.15,random_state=42)

In [ ]:
x_train.shape,x_val.shape, y_train.shape, y_val.shape

In [ ]:
initial_epochs=3
batch_size=32
fine_tune_epochs=40

In [ ]:

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it does something you should see a number as output

    # use the same config as you used to create the session
    config = tensorflow.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.compat.v1.Session(config=config))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(
        width_shift_range=0.2, height_shift_range=0.2,
        horizontal_flip=True,   # randomly flip images
        vertical_flip=True,     # randomly flip images
        rotation_range=60 ,      # Degree range for random rotations
        zoom_range=0.2, shear_range=0.2, fill_mode='nearest')

valid_data_gen = ImageDataGenerator()

# Using original generator
train_generator = train_data_gen.flow(x_train, y_train,batch_size=batch_size,seed=2019)

valid_generator = valid_data_gen.flow(x_val, y_val,batch_size=batch_size,seed=2019)

In [ ]:
fig,ax=plt.subplots(nrows=1,ncols=7,figsize=(18,8))
i = 0
aug = ['width_shift_range','height_shift_range','horizontal_flip','vertical_flip','rotation_range','zoom_range','shear_range']
#for img_iterator in datagen.flow(x = img_arr,batch_size = 1):
for x,y in train_data_gen.flow(x_train, y_train): 
    ax[i].set_title("{}".format(aug[i]), size=16)
    ax[i].imshow((x[0].astype('uint8'))) 
    ax[i].axis('off')  
    
    i = i + 1
    if i == 7:
        break
#fig.savefig("images_pre/Aug_21.png",dpi=96)

In [ ]:
train = pd.read_csv('G/APTOS/train.csv')

In [ ]:
train['image'] = [i + '.png' for i in train['id_code']]

In [ ]:

TRAIN_PATH ='G/APTOS/train-final-256/' 
train['file_path'] = train['id_code'].map(lambda x: os.path.join(TRAIN_PATH,'{}.png'.format(x)))

In [ ]:
fig,ax=plt.subplots(nrows=1,ncols=8,figsize=(30,10))
i = 0
img = cv2.imread(train.file_path.iloc[np.random.randint(low = 0,high = len(train) - 1)])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (256,256))
img_arr = img.reshape((1,) + img.shape)
ax[i].set_title("original_image", size=16)
ax[i].imshow(img)
ax[i].axis("off")
# plt.rcParams["axes.grid"] = False
i = i + 1
aug = ['width_shift_range','height_shift_range','horizontal_flip','vertical_flip','rotation_range','zoom_range','shear_range']
for img_iterator in train_data_gen.flow(x = img_arr,batch_size = 1):
# for x,y in train_data_gen.flow(x_train, y_train): 
    ax[i].set_title("{}".format(aug[i-1]), size=16)
    ax[i].imshow((x[0].astype('uint8'))) 
    ax[i].axis('off')  
    
    i = i + 1
    if i == 8:
        break
#fig.savefig("images_pre/Aug_21_New.png",dpi=96)

## Load in Pretrained Models

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet',include_top=False, input_shape=(256,256,3))
resnet=keras.applications.ResNet50(weights='imagenet',include_top=False, input_shape=(256,256,3)) 
densenet=keras.applications.DenseNet201(weights='imagenet',include_top=False, input_shape=(256,256,3))
inceptnet=keras.applications.InceptionV3(weights='imagenet',include_top=False, input_shape=(256,256,3))
incept_resnet=keras.applications.InceptionResNetV2(weights='imagenet',include_top=False, input_shape=(256,256,3))
xceptnet=keras.applications.Xception(weights='imagenet',include_top=False, input_shape=(256,256,3))

## Define & Train the Model

### Feature-Extraction

If performing fine tuning directly would result in a huge gradient, so it's better that we perform 3 epochs of feature extraction first so that weights of the final fully connected layer aren't completely random. The intuition for this is that if we don't perform feature-extraction, then the gradient will be too large and will change the pretrained weights too much.


In [ ]:
def Initial_Training_1(base_model,name):

    for layer in base_model.layers:
        print(layer.name)
        layer.trainable = False
        
    print(len(base_model.layers))
   

    # Get the output from the base model 
    base_model_ouput = base_model.layers[-1].output

    x = layers.BatchNormalization()(base_model_ouput)
    # Flatten the output layer to 1 dimension
    x = layers.GlobalAveragePooling2D()(x)
    # Add a fully connected layer with 1024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
     # Add a dropout rate of 0.5
    x = layers.Dropout(0.5)(x)
    # Add a fully connected layer with 1024 hidden units and ReLU activation
    x = layers.Dense(512, activation='relu')(x)
    # Add a dropout rate of 0.5
    x = layers.Dropout(0.5)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(5, activation='softmax')(x)

    # Configure and compile the model

    model = Model(inputs=base_model.input, outputs=x,name=name)

    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
                
    model.summary()

    history= model.fit_generator(train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=initial_epochs,verbose=1)
        
    return model, history

In [ ]:
def Initial_Training_2(base_model,name):

  #Reset the batch norm moving averages and allow them to update to the new dataset
  
    for layer in base_model.layers:
        print(layer.name)
        if hasattr(layer, 'moving_mean') and hasattr(layer, 'moving_variance'):
            layer.trainable = True
            K.eval(K.update(layer.moving_mean, K.zeros_like(layer.moving_mean)))
            K.eval(K.update(layer.moving_variance, K.zeros_like(layer.moving_variance)))
        else:
            layer.trainable = False

    print(len(base_model.layers))

    
    # Get the output from the base model 
   
    base_model_ouput = base_model.layers[-1].output

    x = layers.BatchNormalization(name=name+'_BN')(base_model_ouput)
    # Flatten the output layer to 1 dimension
    x = layers.GlobalAveragePooling2D()(x)
    # Add a fully connected layer with 1024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
     # Add a dropout rate of 0.5
    x = layers.Dropout(0.5)(x)
    # Add a fully connected layer with 1024 hidden units and ReLU activation
    x = layers.Dense(512, activation='relu')(x)
    # Add a dropout rate of 0.5
    x = layers.Dropout(0.5)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(5, activation='softmax')(x)

    # Configure and compile the model

    model = Model(inputs=base_model.input, outputs=x,name=name)

    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
    model.summary()

    history= model.fit_generator(train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=initial_epochs,
    verbose=1)
    
    
    return model, history

### Fine Tuning Model

Let's fine tune the last convolutional block of network. I only use learning_rate = 0.0001 with very high momentum = 0.9 and train for 40 epochs only so that the original weights of pretrained network won't be changed too much. learning_rate_reduction function is used and will halve the learning_rate whenever the validation accuracy plateaus for 3 epochs.

In [ ]:
import time
def model_finetuning(model,batch_size,split_at,model_path):

    #epochs = 30

    # training the model after 15 layers
   
    for layer in model.layers[:split_at]: layer.trainable = False
    for layer in model.layers[split_at:]: layer.trainable = True
        
   
    # Choosing lower learning rate for fine-tuning
    # learning rate is generally 10-1000 times lower than normal learning rate when we are fine tuning the initial layers
    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, 
                                                min_lr=0.000001, cooldown=3)
    checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only = True, mode ='auto', verbose = 1)

    batch_size = 32
    epochs = 40
    start_time = time.time()
    history_finetune = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n//batch_size,
        validation_data=valid_generator,
        validation_steps=valid_generator.n//batch_size,
        epochs=fine_tune_epochs,
        verbose=1, callbacks=[learning_rate_reduction,checkpoint])
    print("\n")
    print("Time taken for training the model: {} seconds".format(np.round(time.time()-start_time,5)))
    return model,history_finetune

In [ ]:
def plot_history(model_history,model_name):
    loss_list = [s for s in model_history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in model_history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in model_history.history.keys() if 'accuracy' in s and 'val' not in s]
    val_acc_list = [s for s in model_history.history.keys() if 'accuracy' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(model_history.history[loss_list[0]]) + 1)
    
    ## Loss
    #fig = plt.figure()
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, model_history.history[l], 'b', label='Training loss')
    for l in val_loss_list:
        plt.plot(epochs, model_history.history[l], 'g', label='Validation loss')
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    # plt.legend(['Training set', 'Validation set'], loc='best')
    plt.legend()
    plt.savefig('images/New_'+ model_name+'_loss.jpg',dpi=96)

    ## Accuracy
    plt.figure(2)
   
    for l in acc_list:
        plt.plot(epochs, model_history.history[l], 'b', label='Training accuracy' )
    for l in val_acc_list:    
        plt.plot(epochs, model_history.history[l], 'g', label='Validation accuracy')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    #plt.legend(['Training set', 'Validation set'], loc='best')
    plt.legend()
    plt.savefig('images/New_'+ model_name+'_accuracy.jpg',dpi=96)

    #plt.close(fig)
    plt.show()

In [ ]:
def performace_evaluate(model):

    loss, accuracy = model.evaluate(x_train, y_train, verbose=1)
    loss_v, accuracy_v = model.evaluate(x_val, y_val, verbose=1)
    print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
    print("Train: accuracy = %f  ;  loss = %f" % (accuracy, loss))

    # Make a prediction on the test set
    start_time = time.time()
    y_pred = model.predict(x_val)
    print("\n")
    print("Time taken for predicting the model: {} seconds".format(np.round(time.time()-start_time,5)))
    y_pred1 = np.round(y_pred)

    #y_pred_class = np.argmax(y_pred1,axis = 1)
    y_pred_class = np.argmax(y_pred,axis = 1)

    y_true = np.argmax(y_val,axis = 1) 

    cm = confusion_matrix(y_true,y_pred_class)
    print("\n Confusion Matrix \n")
    print(cm)

    # Check the Classification Report
    print("\n Classification Report \n")
    print(classification_report(y_val, y_pred1))
    
    #def plot_conf_matrix(true,pred,classes):
    labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
    

    df_cm = pd.DataFrame(cm, range(len(labels)), range(len(labels)))
    plt.figure(figsize=(8,5.5))
    sns.set(font_scale=1.4)
    sns.heatmap(df_cm, annot=True, annot_kws={"size": 16},xticklabels = labels ,yticklabels = labels,fmt='g')
    plt.xlabel('Predicted Label')
    plt.ylabel('Actual Label')
    plt.show()

    accuracy= accuracy_score(y_true, y_pred_class)
    print('Accuracy: {}'.format(accuracy))
    precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred_class, average='weighted')
    print('precision: {}'.format(precision))
    print('recall: {}'.format(recall))
    print('fscore: {}'.format(fscore))


    y_predNew = y_pred.astype(int).sum(axis=1) - 1

    #from sklearn.metrics import cohen_kappa_score
    kappa_score = cohen_kappa_score(
                y_true,
                y_pred_class, 
                weights='quadratic'
            )
    print('QWK Score: {}'.format(kappa_score))


    print("\n")

    print("\n Predicting labels using validation dataset\n\n")

    #labels=['Class 0','Class 1','Class 2','Class 3','Class 4']
    labels=['No DR','Mild DR','Moderate DR','Severe DR','PDR']

    columns=4
    rows=3      

    fig, axes = plt.subplots(ncols=columns, nrows=rows, figsize=(12, 8))
    # plt.subplots_adjust(top = 0.9, bottom=0.01, hspace=1.4, wspace=0.1)#
   
    index=0
    for i in range(3):
        for j in range(4):
           
            axes[i,j].set_title("True: {}\nPredicted: {}".format(labels[y_pred_class[index]],labels[y_true[index]]),size=16)
            axes[i,j].imshow(x_val[index], cmap='gray')
            axes[i,j].get_xaxis().set_visible(False)
            axes[i,j].get_yaxis().set_visible(False)
            index += 1

    fig.tight_layout()
    plt.show()



In [ ]:
def test_prediction(model):

    
     # Make a prediction on the test set
    start_time = time.time()
    y_pred = model.predict(x_test)
    print("\n")
    print("Time taken for predicting the model: {} seconds".format(np.round(time.time()-start_time,5)))
    y_pred1 = np.round(y_pred)

    #y_pred_class = np.argmax(y_pred1,axis = 1)
    y_pred_class = np.argmax(y_pred,axis = 1)

    y_true = np.argmax(y_test,axis = 1) 

    cm = confusion_matrix(y_true,y_pred_class)
    print("\n Confusion Matrix \n")
    print(cm)

    # Check the Classification Report
    print("\n Classification Report \n")
    print(classification_report(y_test, y_pred1))
    
    #def plot_conf_matrix(true,pred,classes):
    labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
    

    df_cm = pd.DataFrame(cm, range(len(labels)), range(len(labels)))
    plt.figure(figsize=(8,5.5))
    sns.set(font_scale=1.4)
    sns.heatmap(df_cm, annot=True, annot_kws={"size": 16},xticklabels = labels ,yticklabels = labels,fmt='g')
    plt.xlabel('Predicted Label')
    plt.ylabel('Actual Label')
    plt.show()

    accuracy= accuracy_score(y_true, y_pred_class)
    print('Accuracy: {}'.format(accuracy))
    precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred_class, average='weighted')
    print('precision: {}'.format(precision))
    print('recall: {}'.format(recall))
    print('fscore: {}'.format(fscore))


    y_predNew = y_pred.astype(int).sum(axis=1) - 1

    #from sklearn.metrics import cohen_kappa_score
    kappa_score = cohen_kappa_score(
                y_true,
                y_pred_class, 
                weights='quadratic'
            )
    print('QWK Score: {}'.format(kappa_score))


    print("\n")

    print("\n Predicting labels using validation dataset\n\n")

    #labels=['Class 0','Class 1','Class 2','Class 3','Class 4']
    class_labels=['No DR','Mild DR','Moderate DR','Severe DR','PDR']

    columns=4
    rows=3      

    fig, axes = plt.subplots(ncols=columns, nrows=rows, figsize=(12, 8))
    # fig, axes = plt.subplots(ncols=4, nrows=3, figsize=(16,10))
    # plt.subplots_adjust(top = 0.9, bottom=0.01, hspace=1.4, wspace=0.2)#
 

    index=0
    for i in range(3):
        for j in range(4):
          
            axes[i,j].set_title("True: {}\nPredicted: {}".format(class_labels[y_pred_class[index]],class_labels[y_true[index]]),size=16)
            axes[i,j].imshow(x_val[index], cmap='gray')
            axes[i,j].get_xaxis().set_visible(False)
            axes[i,j].get_yaxis().set_visible(False)
            index += 1

    fig.tight_layout()
    plt.show()




In [ ]:
def predict_labels(model):

    y_test = model.predict(x_test_pred,verbose=1)
    predictions = [np.argmax(pred) for pred in y_test]

    model_to_submit = pd.DataFrame({
        'id_code':test['id_code'],
        'diagnosis':predictions
    })

    print("\n Distribution of predicted label \n")
    import seaborn as sns
    import matplotlib.pyplot as plt
    f, ax = plt.subplots(figsize=(14, 8.7))
    ax = sns.countplot(x="diagnosis", data=model_to_submit, palette="GnBu_d")
    sns.despine()
    ax.set_title("Distribution of predicted label")
    plt.show()


    print("\n")

    print("\n Predicting labels using Test dataset\n\n")

    #def display_samples(df,
    columns=4
    rows=3

    fig=plt.figure(figsize=(5*columns, 4*rows))
    #fig=plt.figure(figsize=(12,8))
    #fig=plt.figure(figsize=(15, 8))
    #plt.subplots_adjust (hspace=1.4)

    for i in range(columns*rows):
        image_path = test.loc[i,'id_code']
        #print(image_path)
        image_id = model_to_submit.loc[i,'diagnosis']
        
        if image_id==0:
            val="No DR"
        elif image_id==1:
            val="Mild DR"
        elif image_id==2:
            val="Moderate DR"
        elif image_id==3:
            val="Severe DR"   
        elif image_id==4:
            val="Proliferative DR"    
        img = cv2.imread(f'G/APTOS/test-final-256/{image_path}.png')
        #print(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        fig.add_subplot(rows, columns, i+1)
        
        plt.title("Image={image_path}.png \nDiagnosis={image_id}".format(image_path=image_path, image_id=val),size=16)
        
        plt.axis('off')
        plt.imshow(img)

# VGG16

In [ ]:
vgg_model,vgg_history= Initial_Training_1(vgg,"Vgg_1")

In [ ]:
vgg_FT,vgg_FT_history=model_finetuning(vgg_model,32,15,'vgg16.h5')
vgg_FT.save('vgg16_Full.h5')

In [ ]:
vgg_FT.summary()

In [ ]:
plot_history(vgg_FT_history,"Vgg16") 

In [ ]:
performace_evaluate(vgg_FT)

##  Make prediction on Unseen Test Dataset

In [ ]:
test_prediction(vgg_FT)

##  Make prediction on Test Dataset


In [ ]:
predict_labels(vgg_FT)

## ResNet50

In [ ]:
resnet_model,resnet_history= Initial_Training_1(resnet,"resnet_1")

In [ ]:
resnet_FT,resnet_FT_history=model_finetuning(resnet_model,32,140,'resnet.h5')
resnet_FT.save('resnet_Full.h5')

In [ ]:
resnet_FT.summary()

In [ ]:
plot_history(resnet_FT_history,"resnet")

In [ ]:
performace_evaluate(resnet_FT)

##  Evaluate Performance on Unseen Test Dataset

In [ ]:
test_prediction(resnet_FT)

##  Evaluate Performance on Unseen Test Dataset

In [ ]:
predict_labels(resnet_FT)

In [ ]:
#gc.collect()
reset_keras()

# DenseNet201

In [ ]:
densenet_model,densenet_history= Initial_Training_1(densenet,"densenet_1")

In [ ]:
densenet_FT,densenet_FT_history=model_finetuning(densenet_model,32,481,'densenet.h5')
densenet_FT.save('densenet_Full.h5')

In [ ]:
densenet_FT.summary()

In [ ]:
plot_history(densenet_FT_history,"densenet")

In [ ]:
# plot_history(densenet_FT_history,,"densenet")

In [ ]:
performace_evaluate(densenet_FT)

##  Evaluate Performance on Unseen Test Dataset



In [ ]:
test_prediction(densenet_FT)

##  Make prediction on Test Dataset


In [ ]:

predict_labels(densenet_FT)

# InceptionV3

In [ ]:

inceptnet_model,incept_history= Initial_Training_2(inceptnet,"incept_1")

In [ ]:
incept_FT,incept_FT_history=model_finetuning(inceptnet_model,32,249,'incept.h5')
incept_FT.save('incept_Full.h5')

In [ ]:
incept_FT.summary()

In [ ]:
plot_history(incept_FT_history,"InceptionV3") 

In [ ]:
# plot_history(incept_FT_history) 

In [ ]:
performace_evaluate(incept_FT)

##  Evaluate Performance on Unseen Test Dataset


In [ ]:
test_prediction(incept_FT)

##  Make prediction on Test Dataset

In [ ]:
predict_labels(incept_FT)

# InceptionResNetV2

In [ ]:
incept_resnet_model, incept_resnet_history= Initial_Training_2(incept_resnet,"incept_res_1")

In [ ]:
incep_res_FT,incep_res_FT_history=model_finetuning(incept_resnet_model,32,618,'incep_res.h5')
incep_res_FT.save('incep_res_Full.h5')

In [ ]:
incep_res_FT.summary()

In [ ]:
plot_history(incep_res_FT_history,"InceptionResNetV2") 

In [ ]:
# plot_history(incep_res_FT_history) 

In [ ]:
performace_evaluate(incep_res_FT)

##  Evaluate Performance on Unseen Test Dataset

In [ ]:
test_prediction(incep_res_FT)

##  Make prediction on Test Dataset

In [ ]:
predict_labels(incep_res_FT)

# Xception

In [ ]:
xceptnet_model,xcept_history= Initial_Training_2(xceptnet,"xcept_1")

In [ ]:
xceptnet_FT,xceptnet_FT_history=model_finetuning(xceptnet_model,32,116,'Xcept.h5')
xceptnet_FT.save('Xcept_Full.h5')

In [ ]:
xceptnet_FT.summary()

In [ ]:
plot_history(xceptnet_FT_history,"Xception") 


In [ ]:
#plot_history(xceptnet_FT_history) 

In [ ]:
performace_evaluate(xceptnet_FT)

##  Evaluate Performance on Unseen Test Dataset

In [ ]:
test_prediction(xceptnet_FT)

##  Make prediction on Test Dataset

In [ ]:
predict_labels(xceptnet_FT)

# Ensemble Model

In [ ]:
# vgg_FT=load_model('vgg16_full.h5')
# resnet_FT=load_model('resnet_full.h5')
# densenet_FT=load_model('densenet_full.h5')
# incept_FT=load_model('incept_full.h5')
# incep_res_FT=load_model('incep_res_full.h5')
# xceptnet_FT=load_model('xcept_full.h5')

In [ ]:
def ensembleModels(models, model_input,name):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=layers.average(yModels) 
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg,name=name)  
   #,    name='ensemble'
    return modelEns

In [ ]:
all_models= [vgg_FT,resnet_FT,densenet_FT,incept_FT,incep_res_FT,xceptnet_FT]
model_input = Input(shape=all_models[0].input_shape[1:]) # c*h*w
modelEns = ensembleModels(all_models, model_input,"Ensemble_All")

In [ ]:
modelEns.summary()

In [ ]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
modelEns.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

In [ ]:
performace_evaluate(modelEns)

In [ ]:
performace_evaluate(modelEns)

In [ ]:
modelEns.save('modelEns_all_full.h5')

##  Evaluate Performance on Unseen Test Dataset

In [ ]:
test_prediction(modelEns)

##  Make prediction on Test Dataset



In [ ]:

predict_labels(modelEns)

### Ensemble model - Except ResNet50

In [ ]:
all_models_R= [vgg_FT,densenet_FT,incept_FT,incep_res_FT,xceptnet_FT]
model_input_R = Input(shape=all_models_R[0].input_shape[1:]) # c*h*w
modelEns_R = ensembleModels(all_models_R, model_input_R,"Ensemble_ExRes")

In [ ]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
modelEns_R.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

In [ ]:
performace_evaluate(modelEns_R)

In [ ]:
modelEns_R.save('modelEns_R.h5')

## Ensemble model - Except DenseNet201

In [ ]:
all_models_D= [vgg_FT,resnet_FT,incept_FT,incep_res_FT,xceptnet_FT]
model_input_D = Input(shape=all_models_D[0].input_shape[1:]) # c*h*w
modelEns_D = ensembleModels(all_models_D, model_input_D,"Ensemble_ExDense")

In [ ]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
modelEns_D.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

In [ ]:
performace_evaluate(modelEns_D)

In [ ]:
test_prediction(modelEns_D)

In [ ]:

predict_labels(modelEns_D)

In [ ]:
modelEns_D.save('modelEns_D.h5')

In [ ]:
y_test_pred = modelEns.predict(x_test_pred,verbose=1)
predictions = [np.argmax(pred) for pred in y_test_pred]

In [ ]:
test = pd.read_csv('C:/Users/fredd/Downloads/aptos2019-blindness-detection/test.csv')

In [ ]:
test['diagnosis'] = predictions
test.to_csv('modelEnsemble.csv',index=False)
test

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(14, 8.7))
ax = sns.countplot(x="diagnosis", data=test, palette="GnBu_d")
sns.despine()
plt.show()

In [ ]:
#def display_samples(df,
columns=4
rows=3

fig=plt.figure(figsize=(5*columns, 4*rows))
#fig=plt.figure(figsize=(15, 8))
for i in range(columns*rows):
    image_path = test.loc[i,'id_code']
    #print(image_path)
    image_id = test.loc[i,'diagnosis']
    if image_id==0:
        val="No DR"
    elif image_id==1:
        val="Mild DR"
    elif image_id==2:
        val="Moderate DR"
    elif image_id==3:
        val="Severe DR"   
    elif image_id==4:
        val="Proliferative DR"    
    img = cv2.imread(f'G/APTOS/test-final-256/{image_path}.png')
    #print(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    fig.add_subplot(rows, columns, i+1)
    #plt.title(["Image:" image_path "Label = "+image_id])
    plt.title("Image={image_path}.png \nDiagnosis={image_id}".format(image_path=image_path, image_id=val))
    #plt.title(image_path)
    #plt.title(image_id)
    plt.axis('off')
    plt.imshow(img)

### COMPARISON WITH OTHER WORKS

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


labels = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
model = [97.8, 97.8, 97.8, 97.8]
model_qumar = [80.8, 63.8, 51.5, 53.7]

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars


#fig, ax = plt.subplots()
fig, ax = plt.subplots(figsize=(8, 5))
rects1 = ax.bar(x - width/2, model, width, label='proposed model')
rects2 = ax.bar(x + width/2, model_qumar, width, label='Qummar et al')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Comparison with S.Qummar model')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(loc='center')


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}%'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


labels = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
model = [97.8, 97.8, 97.8, 97.8]
model_qumar = [91, 90.4, 89.54, 89.97]

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars


#fig, ax = plt.subplots()
fig, ax = plt.subplots(figsize=(8, 5))
rects1 = ax.bar(x - width/2, model, width, label='proposed model')
rects2 = ax.bar(x + width/2, model_qumar, width, label='Sikder et al')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Comparison with Niloy Sikder model')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(loc='center')


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}%'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()